# Coin Change Problem

**Note: This problem has multiple solutions and is a classic problem in showing issues with basic recursion. There are better solutions involving memoization and simple iterative solutions.If you are having trouble with this problem (or it seems to be taking a long time to run in some cases) check out the Solution Notebook and fully read the conclusion link for a detailed description of the various ways to solve this problem!**


This problem is common enough that is actually has its own [Wikipedia Entry](https://en.wikipedia.org/wiki/Change-making_problem)! Let's check the problem statement again:

This is a classic recursion problem: Given a target amount **n** and a list (array) of distinct coin values, what's the fewest coins needed to make the change amount. 

For example:

If n = 10 and coins = [1,5,10]. Then there are 4 possible ways to make change:

* 1+1+1+1+1+1+1+1+1+1

* 5 + 1+1+1+1+1

* 5+5

* 10

With 1 coin being the minimum amount.

    
## Solution

This is a classic problem to show the value of dynamic programming. We'll show a basic recursive example and show why it's actually not the best way to solve this problem.

Make sure to read the comments in the code below to fully understand the basic logic!

In [1]:
def rec_coin(target,coins):
    '''
    INPUT: Target change amount and list of coin values
    OUTPUT: Minimum coins needed to make change
    
    Note, this solution is not optimized.
    '''
    
    # Default to target value
    min_coins = target
    
    # Check to see if we have a single coin match (BASE CASE)
    if target in coins:
        return 1
    
    else:
        
        # For every coin value that is <= than target
        # With this approach we explore all coin combinations,
        # but we should try to subtract large coin values first..
        for i in [c for c in coins if c <= target]:
            
            # Recursive Call (add a count coin and subtract from the target) 
            num_coins = 1 + rec_coin(target-i,coins)
            
            # Reset Minimum if we have a new minimum
            if num_coins < min_coins:
                
                min_coins = num_coins
                
    return min_coins

Let's see it in action.

In [2]:
# It takes a very long time!
rec_coin(63,[1,5,10,25])

6

The problem with this approach is that it is very inefficient! It can take many, many recursive calls to finish this problem and its also inaccurate for non standard coin values (coin values that are not 1,5,10, etc.)

We can see the problem with this approach in the figure below:

In [4]:
from IPython.display import Image
Image(url='http://interactivepython.org/runestone/static/pythonds/_images/callTree.png')

Each node here corresponds to a call to the **rec_coin** function. The label on the node indicated the amount of change for which we are now computng the number of coins for. Note how we are recalculating values we've already solved! For instance 15 is called 3 times. It would be much better if we could keep track of function calls we've already made.
_____
## Dynamic Programming Solution

This is the key to reducing the work time for the function. The better solution is to remember past results, that way before computing a new minimum we can check to see if we already know a result.

Let's implement this:

In [8]:
def rec_coin_dynam(target,coins,known_results):
    '''
    INPUT: This funciton takes in a target amount and a list of possible coins to use.
    It also takes a third parameter, known_results, indicating previously calculated results.
    The known_results parameter shoud be started with [0] * (target+1)
    
    OUTPUT: Minimum number of coins needed to make the target.
    '''
    
    # Default output to target
    min_coins = target
    
    # Base Case
    if target in coins:
        known_results[target] = 1
        return 1
    
    # Return a known result if it happens to be greater than 1
    elif known_results[target] > 0:
        return known_results[target]
    
    else:
        # for every coin value that is <= than target
        for i in [c for c in coins if c <= target]:
            
            # Recursive call, note how we include the known results!
            num_coins = 1 + rec_coin_dynam(target-i,coins,known_results)
            
            # Reset Minimum if we have a new minimum
            if num_coins < min_coins:
                min_coins = num_coins
                
                # Reset the known result
                known_results[target] = min_coins
                
    return min_coins

Let's test it!

In [9]:
target = 74
coins = [1,5,10,25]
known_results = [0]*(target+1)

rec_coin_dynam(target,coins,known_results)

8

## My Implementation

In [28]:
def min_coins(n, coins):
    
    # A recursive function to find the minimum number of coins needed to make the value n
    def min_coins_recursive(n):
        # If the value is 0, no coins are needed to make it
        if n == 0:
            return 0

        # If the value has been computed before, return the stored result
        if n in min_coins_memo:
            return min_coins_memo[n]

        # Set the initial minimum number of coins needed to the maximum possible value
        # Since coins can have minimum value 1, min_coins_memo[n] could be n itself!
        min_coins_memo[n] = float("inf")

        # Iterate through each coin value
        for coin in coins:
            # If the current coin value is less than or equal to the current value
            if coin <= n:
                # Update the minimum number of coins needed for the current value
                # Since we use the current coin, we subtract it to the target n
                # and increase the coin count with the current coin.
                # Using min() is the same as computing the recursive call
                # and updating min_coins_memo is the result is smaller than what we've got.
                min_coins_memo[n] = min(min_coins_memo[n], min_coins_recursive(n - coin) + 1)

        # Return the minimum number of coins needed for the current value
        return min_coins_memo[n]

    # Initialize a dictionary to store the results of expensive function calls
    min_coins_memo = {}
    
    # Call the recursive function to find the minimum number of coins needed to make the target value
    result = min_coins_recursive(n)

    # Return the result
    return result


In [29]:
min_coins(63,[1,5,10,25])

6

### Re-Implementation

Given a target amount n and a list (array) of distinct coin values, what's the fewest coins needed to make the change amount.

If n = 10 and coins = [1,5,10]. Then there are 4 possible ways to make change:

* 1+1+1+1+1+1+1+1+1+1

* 5 + 1+1+1+1+1

* 5+5

* 10

With 1 coin being the minimum amount.

In [60]:
def coin_change(n, coins):
    min_coins = n
    for coin in coins:
        if coin <= n:
            rest = n%coin
            k = (n - rest)//coin
            if rest > 0:
                # rest is non 0, recusion
                k += coin_change(rest, coins)
            if k < min_coins:
                min_coins = k
    return min_coins

In [61]:
coins = [1,5,10,25]
coin_change(45,coins)

3

In [66]:
class Memoize:
    def __init__(self, function):
        self.function = function
        self.memo = dict()
    def __call__(self, *args):
        if not args[0] in self.memo:
            self.memo[args[0]] = self.function(*args)
        return self.memo[args[0]]

In [69]:
coin_change_mem = Memoize(coin_change)

In [70]:
def coin_change_(n, coins):
    def coin_change_rec(n, coins):
        min_coins = n
        for coin in coins:
            if coin <= n:
                rest = n%coin
                k = (n - rest)//coin
                if rest > 0:
                    # rest is non 0, recusion
                    k += coin_change(rest, coins)
                if k < min_coins:
                    min_coins = k
        return min_coins
    memo = dict()
    if not n in memo:
        memo[n] = coin_change_rec(n, coins)
    return memo[n]
        

# Test Your Solution

Run the cell below to test your function against some test cases. 

**Note that the TestCoins class only test functions with two parameter inputs, the list of coins and the target**

In [71]:
"""
RUN THIS CELL TO TEST YOUR FUNCTION.
NOTE: NON-DYNAMIC FUNCTIONS WILL TAKE A LONG TIME TO TEST. IF YOU BELIEVE YOU HAVE A SOLUTION
"""

class TestCoins(object):
    
    def check(self,solution):
        coins = [1,5,10,25]
        assert (solution(45,coins) == 3)
        assert (solution(23,coins) == 5)
        assert (solution(74,coins) == 8)

        print('Passed all tests.')
        
# Run Test

test = TestCoins()
#test.check(rec_coin)
test.check(min_coins)
test.check(coin_change)
test.check(coin_change_mem)
test.check(coin_change_)


Passed all tests.
Passed all tests.
Passed all tests.
Passed all tests.


# Conclusion and Extra Resources

For homework, read the link below and also implement the non-recursive solution described in the link!

For another great resource on a variation of this problem, check out this link:
[Dynamic Programming Coin Change Problem](http://interactivepython.org/runestone/static/pythonds/Recursion/DynamicProgramming.html)